In [102]:
#Import Libraries
import numpy as np
from nltk import sent_tokenize, word_tokenize
import nltk
import pandas as pd
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.corpus import sentiwordnet as swn
import re
from textblob import TextBlob
import keras
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from functools import reduce

In [103]:
d1 = pd.read_csv('all_hotel_attrs_bookingcom.csv')
d2 = pd.read_csv('all_hotel_attrs_tripadvisor.csv')
d3 = pd.read_csv('Combined_Hotels_Summary (4).csv')
d3

,Hotel_id,Bookingcom_hotel_id,Booking_com_rating,reviews_bookingcom,top_positive_features,top_negative_features,top_talked_about_features,Oldest_review_date,latest_review_date,common_surrounding_hotels_features,Composite_rating,Tripadvisor_hotel_id,Trip_advisor_rating,review_tripadvisor,Hotels_com_rating,different_surrounding_hotels_features,Hotels_combined_rating,Room_type
0,0,4209583.0,0.937500,75 reviews,"[[('bathroom', 8), ('room', 6), ('que', 3), ('...","[[('y', 14), ('breakfast', 7), ('location', 7)...",[[('exceptionalexceptionalexceptionalexception...,12/11/2018,25/06/2019,"['Free WiFi Internet Access Included', 'Non Sm...",0.937500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,49918.0,0.781250,"1,699 reviews","[[('room', 25), ('hotel', 11), ('night', 9), (...","[[('location', 57), ('staff', 34), ('room', 23...","[[('location', 10), ('hotel', 8), ('value', 4)...",13/11/2018,24/06/2019,"['Free WiFi Internet Access Included', 'Non Sm...",0.781250,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,92275.0,0.734375,"4,023 reviews","[[('room', 34), ('breakfast', 14), ('hotel', 1...","[[('location', 79), ('breakfast', 30), ('staff...","[[('location', 19), ('value', 8), ('staff', 8)...",15/07/2017,25/06/2019,"['Free WiFi Internet Access Included', 'Family...",0.734375,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,247627.0,0.609375,"1,150 reviews","[[('room', 29), ('breakfast', 27), ('air', 12)...","[[('location', 42), ('beach', 35), ('staff', 2...","[[('location', 7), ('value', 4), ('barcelona',...",3/7/2017,26/06/2019,"['Parking (fee required)', 'Free WiFi Internet...",0.609375,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1949632.0,0.500000,241 reviews,"[[('room', 24), ('breakfast', 21), ('hotel', 1...","[[('room', 22), ('location', 22), ('staff', 17...","[[('hotel', 7), ('place', 2), ('stay', 2), ('h...",30/06/2017,25/06/2019,"['Family Rooms', 'Free Parking', 'Free WiFi In...",0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,1650,12548113.0,0.700000,28 reviews,[[]],[[]],[[]],None,None,"['Free WiFi Internet Access Included', 'Airpor...",0.703333,2411411.0,0.706667,560 reviews,NaN,NaN,NaN,NaN
5784,1651,301802.0,0.800000,"5,848 reviews",[[]],[[]],[[]],None,None,"['Free WiFi Internet Access Included', 'Swimmi...",0.786667,237290.0,0.773333,"2,326 reviews",NaN,NaN,NaN,NaN
5785,1652,10061785.0,0.700000,3 reviews,[[]],[[]],[[]],None,None,"['Free WiFi Internet Access Included', 'Free P...",0.736667,4633016.0,0.773333,85 reviews,NaN,NaN,NaN,NaN
5786,1653,12640657.0,0.600000,14 reviews,[[]],[[]],[[]],None,None,"['Free WiFi Internet Access Included', 'Airpor...",0.660000,2563824.0,0.720000,64 reviews,NaN,NaN,NaN,NaN


In [4]:
stopwords = [ "hmm","[","]",'·','·',"...","ok","th","rd","th","nd","!","#","$","%","&","'","’","u","v","b","r","(",")","*","+","-",".","/",":",";","<","=",">","?","@","[","/","]","^","_","`","{","|","}","~",",","a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she","please","sorry","bye", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves","hello"]

In [103]:
name = data3['hotel_name']
name  = name.unique()
leng = len(name)

In [106]:
diff_50 = []
diff_30 = []
diff_80 = []
for k in range(0,leng):
    a = name[k]
    hotel3 = data3.loc[data3['hotel_name'] ==a]
    fac = hotel3['facilities_x']
    fac.append(hotel3['facilities_y'])
    city = hotel3['hotel_city_x']
    city = city.iloc[0]
    loc = data3.loc[data3['hotel_city_x'] == city]
    loc = loc.loc[loc['hotel_name'] != a]
    faci = loc['facilities_x']
    faci.append(loc['facilities_y'])
    fac = fac.iloc[0]
    fac = str(fac)
    fac = fac.split(',')
    li1 = []
    for l in fac:
        l = l.replace("'","")
        l = l.replace("[","")
        l = l.replace("]","")
        li1.append(l)
    faci = list(faci)
    li2 = []
    for i in faci:
        i = str(i)
        i = i.split(',')
        for l1 in i:
            l1 = l1.replace("'","")
            l1 = l1.replace("[","")
            l1 = l1.replace("]","")
            li2.append(l1)
    length = len(faci)
    per_50 = 50/100*(length)
    per_30 = 30/100*(length)
    per_80 = 80/100*(length)
    d50 = []
    d30 = []
    d80 = []
    for u in li1:
        v = li2.count(u)
        if(v>per_80):
            d80.append(u)
            continue;
        if(v>per_50):
            d80.append(u)
            continue;
        if(v>per_30):
            d80.append(u)
            continue;
    diff_50.append(d50)
    diff_30.append(d30)
    diff_80.append(d80)
    


In [5]:
d2

,hotel_id,hotel_city,hotel_name,hotel_address,location,facilities,number_of_rooms,price_range,no_of_reviews,rank,rating,rating_class,other_ratings,also_known_as,formerly_known_as
0,property_1657415,London,Park Plaza Westminster Bridge London,"200 Westminster Bridge Road, London SE1 7UT, E...",United KingdomEnglandLondonSouthbank,"['Room service', 'Free Internet', 'Restaurant'...",1019,$207 - $520 (Based on Average Rates for a Stan...,"20,864 reviews",#309,4.5,Excellent,"{'Location': 5.0, 'Cleanliness': 4.5, 'Service...",park plaza westminster bridge london hotel london,NaN
1,property_193143,London,Park Grand London Kensington,"33-37 Hogarth Road | Kensington, London SW5 0Q...",United KingdomEnglandLondonEarls CourtSouth Ke...,"['Room service', 'Restaurant', 'Fitness Center...",133,$159 - $356 (Based on Average Rates for a Stan...,"4,306 reviews",#34,4.5,Excellent,"{'Location': 4.5, 'Cleanliness': 4.5, 'Service...",best western premier shaftsbury london kensing...,BEST WESTERN PREMIER Park Grand London Kensington
2,property_193097,London,The Tower Hotel,"St Katharine's Way, London E1W 1LD, England",United KingdomEnglandLondon,"['Restaurant', 'Room service', 'Free High Spee...",801,$125 - $332 (Based on Average Rates for a Stan...,"9,370 reviews",#394,4,Very good,"{'Location': 4.5, 'Cleanliness': 4.0, 'Service...",the tower hotel londonthe tower londonhotel th...,NaN
3,property_243430,London,London House Hotel,"81 Kensington Gardens Square | Bayswater, Lond...",United KingdomEnglandLondonNotting Hill,"['Free High Speed Internet (WiFi)', 'Breakfast...",103,$119 - $266 (Based on Average Rates for a Stan...,"2,733 reviews",#345,4,Very good,"{'Location': 4.5, 'Cleanliness': 4.5, 'Service...",london house hotellondon hotel,NaN
4,property_13474415,London,Z Hotel Tottenham Court Road,"52 Poland Street, London W1F 7NQ, England",United KingdomEnglandLondonSoho,"['Free High Speed Internet (WiFi)', 'Concierge...",120,$120 - $312 (Based on Average Rates for a Stan...,111 reviews,#389,4.5,Excellent,"{'Location': 5.0, 'Cleanliness': 5.0, 'Service...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10013,property_2525982,Manila,Bukingham Pension,"General Malvar St, Malate, Manila, Luzon, Phil...",PhilippinesLuzonMetro ManilaManila,[],NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN
10014,property_3786493,Manila,Butiki Planet,"CRL Building, Parkhomes, Tunasan, Manila, Luzo...",PhilippinesLuzonMetro ManilaManila,[],2,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN
10015,property_6946138,Manila,Casablanca Manila Guest House,"11-A and 15-A Escarlata Street, Manila, Luzon,...",PhilippinesLuzonMetro ManilaManila,[],2,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN
10016,property_1197748,Manila,Center Point Hotel,"1430 A. Mabini st. Ermita, Manila, Luzon, Phil...",PhilippinesLuzonMetro ManilaManila,[],257,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN


In [104]:
d2["hotel_id"] = d2["hotel_id"].str.replace("property_","")

In [7]:
d1

,hotel_id,hotel_city,hotel_name,hotel_address,loc_coordinates,facilities,room_types,no of reviews,rating,rating class,Other_ratings,hotel_type
0,show_id294936,London,Princes Gardens,"Imperial College London, Prince's Garden, Watt...","51.49864990,-0.17341465","['Free WiFi Internet Access Included', 'Parkin...","['Single Room', 'Double Room', 'Twin Room']","5,298 reviews",8.1,Very good,"{'Staff': '8.0', 'Facilities': '7.4', 'Cleanli...",Student accommodation
1,show_id3690340,London,Beautiful Notting Hill Garden Apartment,"61B Cornwall Crescent, Kensington and Chelsea,...","51.51423600,-0.21119675","['Free WiFi Internet Access Included', 'Non Sm...",['Apartment'],8 reviews,9.5,Exceptional,"{'Staff\xa0': '10', 'Facilities\xa0': '9.7', '...",Apartment
2,show_id4916754,London,"Boutique 2 Bed in Notting Hill, Westbourne Park","19 Aldridge Road Villas Flat 4, Westminster Bo...","51.51918640,-0.20014320","['Family Rooms', 'Heating']",['Two-Bedroom Apartment'],NaN,NaN,NaN,NaN,Apartment
3,show_id5048582,London,-30% NATURAL HISTORY MUSEUM LUXURY APARTMENT,"53 Queen's Gate apartment 4, Kensington and Ch...","51.49640050,-0.17960130","['Free WiFi Internet Access Included', 'Family...",['Two-Bedroom Apartment'],NaN,NaN,NaN,NaN,Apartment
4,show_id4173097,London,The Lovely Longridge Road Apartment,"Longridge Road, Kensington and Chelsea, London...","51.49262000,-0.19841000","['Free WiFi Internet Access Included', 'Heating']",['Studio Apartment'],NaN,NaN,NaN,NaN,Apartment
...,...,...,...,...,...,...,...,...,...,...,...,...
5877,show_id2995477,Manila,Frank Lloyd Apartelle,"Frank Lloyd Mansion Building, Meekness Extensi...","14.48928831,121.01532054",['Free WiFi Internet Access Included'],"['Studio Apartment', 'Studio Apartment', 'Stud...",517 reviews,9.0,Superb,"{'Staff\xa0': '9.7', 'Facilities': '8.6', 'Cle...",Apartment
5878,show_id2222765,Manila,Marites Apartments,"Shell Residences, EDSA corner Sunrise Drive, P...","14.53649887,120.98301172","['Free WiFi Internet Access Included', 'Swimmi...","['Apartment with Balcony', 'Apartment with Bal...",493 reviews,8.5,Very good,"{'Staff': '8.4', 'Facilities': '8.2', 'Cleanli...",Apartment
5879,show_id4452923,Manila,"Carlos P Garcia, C5 Ave.","Carlos P Garcia, C5 Ave. Ridgewood Towers, 163...","14.54116713,121.05505518","['Free WiFi Internet Access Included', 'Swimmi...",['Two-Bedroom Apartment'],NaN,NaN,NaN,NaN,Apartment
5880,show_id1338347,Manila,Novotel Manila Araneta Center,"General Aguinaldo Avenue, Araneta Center, Cuba...","14.62015498,121.05401539","['Free WiFi Internet Access Included', 'Swimmi...","['Deluxe King Room', 'Executive King Room', 'S...","1,765 reviews",8.9,Fabulous,"{'Staff': '8.9', 'Facilities\xa0': '9.0', 'Cle...",Hotel


In [105]:
d1["hotel_id"] = d1["hotel_id"].str.replace("show_id","")

In [10]:
d3

,Hotel_id,Bookingcom_hotel_id,Booking_com_rating,reviews_bookingcom,top_positive_features,top_negative_features,top_talked_about_features,Oldest_review_date,latest_review_date,common_surrounding_hotels_features,Composite_rating,Tripadvisor_hotel_id,Trip_advisor_rating,review_tripadvisor,Hotels_com_rating,different_surrounding_hotels_features,Hotels_combined_rating,Room_type
0,0,4209583.0,0.937500,75 reviews,"[[('bathroom', 8), ('room', 6), ('que', 3), ('...","[[('y', 14), ('breakfast', 7), ('location', 7)...",[[('exceptionalexceptionalexceptionalexception...,12/11/2018,25/06/2019,"['Free WiFi Internet Access Included', 'Non Sm...",0.937500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,49918.0,0.781250,"1,699 reviews","[[('room', 25), ('hotel', 11), ('night', 9), (...","[[('location', 57), ('staff', 34), ('room', 23...","[[('location', 10), ('hotel', 8), ('value', 4)...",13/11/2018,24/06/2019,"['Free WiFi Internet Access Included', 'Non Sm...",0.781250,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,92275.0,0.734375,"4,023 reviews","[[('room', 34), ('breakfast', 14), ('hotel', 1...","[[('location', 79), ('breakfast', 30), ('staff...","[[('location', 19), ('value', 8), ('staff', 8)...",15/07/2017,25/06/2019,"['Free WiFi Internet Access Included', 'Family...",0.734375,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,247627.0,0.609375,"1,150 reviews","[[('room', 29), ('breakfast', 27), ('air', 12)...","[[('location', 42), ('beach', 35), ('staff', 2...","[[('location', 7), ('value', 4), ('barcelona',...",3/7/2017,26/06/2019,"['Parking (fee required)', 'Free WiFi Internet...",0.609375,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1949632.0,0.500000,241 reviews,"[[('room', 24), ('breakfast', 21), ('hotel', 1...","[[('room', 22), ('location', 22), ('staff', 17...","[[('hotel', 7), ('place', 2), ('stay', 2), ('h...",30/06/2017,25/06/2019,"['Family Rooms', 'Free Parking', 'Free WiFi In...",0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,1650,12548113.0,0.700000,28 reviews,[[]],[[]],[[]],None,None,"['Free WiFi Internet Access Included', 'Airpor...",0.703333,2411411.0,0.706667,560 reviews,NaN,NaN,NaN,NaN
5784,1651,301802.0,0.800000,"5,848 reviews",[[]],[[]],[[]],None,None,"['Free WiFi Internet Access Included', 'Swimmi...",0.786667,237290.0,0.773333,"2,326 reviews",NaN,NaN,NaN,NaN
5785,1652,10061785.0,0.700000,3 reviews,[[]],[[]],[[]],None,None,"['Free WiFi Internet Access Included', 'Free P...",0.736667,4633016.0,0.773333,85 reviews,NaN,NaN,NaN,NaN
5786,1653,12640657.0,0.600000,14 reviews,[[]],[[]],[[]],None,None,"['Free WiFi Internet Access Included', 'Airpor...",0.660000,2563824.0,0.720000,64 reviews,NaN,NaN,NaN,NaN


In [44]:
def normalize(lst):
    s = sum(lst)
    return map(lambda x: float(x)/s, lst)

In [4]:
r1 = data1['Booking_com_rating']
r2 = data2['Trip_advisor_rating']
norm1 = [(float(i)-min(r1))/(max(r1)-min(r1)) for i in r1]
norm2 = [(float(i)-min(r2))/(max(r2)-min(r2)) for i in r2]
data1['Booking_com_rating'] = norm1
data2['Trip_advisor_rating'] = norm2
data1['Composite_rating'] = norm1
data2['Composite_rating'] = norm2
data2

,Hotel_id,Tripadvisor_hotel_id,Trip_advisor_rating,review_tripadvisor,top_positive_features,top_negative_features,top_talked_about_features,Oldest_review_date,latest_review_date,common_surrounding_hotels_features,Composite_rating
0,1,636140,0.875,"1,415 reviews","[[('nice', 2)]]",[[]],"[[('hotel', 13), ('location', 10), ('stay', 9)...",1/7/2018,1/5/2019,"['Pool', 'Free parking', 'Fitness Center with ...",0.875
1,2,12918864,0.875,135 reviews,"[[('excellent', 1), ('pleasant', 1)]]","[[('dirty', 1)]]","[[('hotel', 11), ('service', 5), ('burj', 5), ...",1/9/2017,1/5/2019,"['Pool', 'Free parking', 'Restaurant', 'Free H...",0.875
2,3,302457,0.875,"5,631 reviews",[[]],"[[('half', 1)]]","[[('hotel', 5), ('al', 5), ('luxury', 4), ('se...",1/4/2019,1/5/2019,"['Pool', 'Room service', 'Restaurant', 'Free H...",0.875
3,4,9704275,0.875,"2,262 reviews",[[]],[[]],"[[('hotel', 25), ('stay', 12), ('dubai', 10), ...",1/4/2019,1/5/2019,"['Pool', 'Room service', 'Free parking', 'Rest...",0.875
4,5,674662,0.750,621 reviews,"[[('superb', 1), ('okay', 1)]]",[[]],"[[('location', 13), ('hotel', 9), ('stay', 6),...",1/4/2017,1/5/2019,"['Pool', 'Room service', 'Restaurant', 'Fitnes...",0.750
...,...,...,...,...,...,...,...,...,...,...,...
2893,2894,1015046,0.875,"7,668 reviews","[[('perfect', 1), ('fantastic', 1)]]",[[]],"[[('hotel', 12), ('stay', 9), ('family', 5), (...",1/5/2019,1/5/2019,"['Restaurant', 'Bar/Lounge', 'Room service', '...",0.875
2894,2895,15741916,1.000,81 reviews,"[[('beautiful', 1)]]",[[]],"[[('hotel', 11), ('plaza', 5), ('place', 4), (...",1/3/2019,1/5/2019,"['Pool', 'Room service', 'Free parking', 'Rest...",1.000
2895,2896,5667617,0.750,"1,606 reviews","[[('excellent', 1)]]","[[('center', 1)]]","[[('hotel', 12), ('business', 4), ('service', ...",1/4/2018,1/5/2019,"['Pool', 'Room service', 'Free parking', 'Rest...",0.750
2896,2897,739160,0.875,192 reviews,"[[('excellent', 1), ('superb', 1)]]","[[('toilet', 1), ('center', 1)]]","[[('place', 8), ('location', 6), ('service', 5...",1/5/2016,1/5/2019,"['Pool', 'Fitness Center with Gym / Workout Ro...",0.875


In [5]:
res = data1.append(data2,sort=False)

In [6]:
res

,Hotel_id,Bookingcom_hotel_id,Booking_com_rating,reviews_bookingcom,top_positive_features,top_negative_features,top_talked_about_features,Oldest_review_date,latest_review_date,common_surrounding_hotels_features,Composite_rating,Tripadvisor_hotel_id,Trip_advisor_rating,review_tripadvisor
0,1,4209583.0,0.937500,75 reviews,"[[('bathroom', 8), ('room', 6), ('que', 3), ('...","[[('y', 14), ('breakfast', 7), ('location', 7)...",[[('exceptionalexceptionalexceptionalexception...,12/11/2018,25/06/2019,"['Free WiFi Internet Access Included', 'Non Sm...",0.937500,NaN,NaN,NaN
1,2,49918.0,0.781250,"1,699 reviews","[[('room', 25), ('hotel', 11), ('night', 9), (...","[[('location', 57), ('staff', 34), ('room', 23...","[[('location', 10), ('hotel', 8), ('value', 4)...",13/11/2018,24/06/2019,"['Free WiFi Internet Access Included', 'Non Sm...",0.781250,NaN,NaN,NaN
2,3,92275.0,0.734375,"4,023 reviews","[[('room', 34), ('breakfast', 14), ('hotel', 1...","[[('location', 79), ('breakfast', 30), ('staff...","[[('location', 19), ('value', 8), ('staff', 8)...",15/07/2017,25/06/2019,"['Free WiFi Internet Access Included', 'Family...",0.734375,NaN,NaN,NaN
3,4,247627.0,0.609375,"1,150 reviews","[[('room', 29), ('breakfast', 27), ('air', 12)...","[[('location', 42), ('beach', 35), ('staff', 2...","[[('location', 7), ('value', 4), ('barcelona',...",3/7/2017,26/06/2019,"['Parking (fee required)', 'Free WiFi Internet...",0.609375,NaN,NaN,NaN
4,5,1949632.0,0.500000,241 reviews,"[[('room', 24), ('breakfast', 21), ('hotel', 1...","[[('room', 22), ('location', 22), ('staff', 17...","[[('hotel', 7), ('place', 2), ('stay', 2), ('h...",30/06/2017,25/06/2019,"['Family Rooms', 'Free Parking', 'Free WiFi In...",0.500000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2893,2894,NaN,NaN,NaN,"[[('perfect', 1), ('fantastic', 1)]]",[[]],"[[('hotel', 12), ('stay', 9), ('family', 5), (...",1/5/2019,1/5/2019,"['Restaurant', 'Bar/Lounge', 'Room service', '...",0.875000,1015046.0,0.875,"7,668 reviews"
2894,2895,NaN,NaN,NaN,"[[('beautiful', 1)]]",[[]],"[[('hotel', 11), ('plaza', 5), ('place', 4), (...",1/3/2019,1/5/2019,"['Pool', 'Room service', 'Free parking', 'Rest...",1.000000,15741916.0,1.000,81 reviews
2895,2896,NaN,NaN,NaN,"[[('excellent', 1)]]","[[('center', 1)]]","[[('hotel', 12), ('business', 4), ('service', ...",1/4/2018,1/5/2019,"['Pool', 'Room service', 'Free parking', 'Rest...",0.750000,5667617.0,0.750,"1,606 reviews"
2896,2897,NaN,NaN,NaN,"[[('excellent', 1), ('superb', 1)]]","[[('toilet', 1), ('center', 1)]]","[[('place', 8), ('location', 6), ('service', 5...",1/5/2016,1/5/2019,"['Pool', 'Fitness Center with Gym / Workout Ro...",0.875000,739160.0,0.875,192 reviews


In [7]:
res1 = res.append(data3,sort=False)

In [8]:
res1

,Hotel_id,Bookingcom_hotel_id,Booking_com_rating,reviews_bookingcom,top_positive_features,top_negative_features,top_talked_about_features,Oldest_review_date,latest_review_date,common_surrounding_hotels_features,Composite_rating,Tripadvisor_hotel_id,Trip_advisor_rating,review_tripadvisor,Hotels_com_rating,different_surrounding_hotels_features,Hotels_combined_rating,Room_type
0,1,4209583.0,0.937500,75 reviews,"[[('bathroom', 8), ('room', 6), ('que', 3), ('...","[[('y', 14), ('breakfast', 7), ('location', 7)...",[[('exceptionalexceptionalexceptionalexception...,12/11/2018,25/06/2019,"['Free WiFi Internet Access Included', 'Non Sm...",0.937500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,49918.0,0.781250,"1,699 reviews","[[('room', 25), ('hotel', 11), ('night', 9), (...","[[('location', 57), ('staff', 34), ('room', 23...","[[('location', 10), ('hotel', 8), ('value', 4)...",13/11/2018,24/06/2019,"['Free WiFi Internet Access Included', 'Non Sm...",0.781250,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,92275.0,0.734375,"4,023 reviews","[[('room', 34), ('breakfast', 14), ('hotel', 1...","[[('location', 79), ('breakfast', 30), ('staff...","[[('location', 19), ('value', 8), ('staff', 8)...",15/07/2017,25/06/2019,"['Free WiFi Internet Access Included', 'Family...",0.734375,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,247627.0,0.609375,"1,150 reviews","[[('room', 29), ('breakfast', 27), ('air', 12)...","[[('location', 42), ('beach', 35), ('staff', 2...","[[('location', 7), ('value', 4), ('barcelona',...",3/7/2017,26/06/2019,"['Parking (fee required)', 'Free WiFi Internet...",0.609375,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1949632.0,0.500000,241 reviews,"[[('room', 24), ('breakfast', 21), ('hotel', 1...","[[('room', 22), ('location', 22), ('staff', 17...","[[('hotel', 7), ('place', 2), ('stay', 2), ('h...",30/06/2017,25/06/2019,"['Family Rooms', 'Free Parking', 'Free WiFi In...",0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1650,1141,12548113.0,0.700000,28 reviews,[[]],[[]],[[]],None,None,"['Free WiFi Internet Access Included', 'Airpor...",0.703333,2411411.0,0.706667,560 reviews,NaN,NaN,NaN,NaN
1651,1142,301802.0,0.800000,"5,848 reviews",[[]],[[]],[[]],None,None,"['Free WiFi Internet Access Included', 'Swimmi...",0.786667,237290.0,0.773333,"2,326 reviews",NaN,NaN,NaN,NaN
1652,1143,10061785.0,0.700000,3 reviews,[[]],[[]],[[]],None,None,"['Free WiFi Internet Access Included', 'Free P...",0.736667,4633016.0,0.773333,85 reviews,NaN,NaN,NaN,NaN
1653,1144,12640657.0,0.600000,14 reviews,[[]],[[]],[[]],None,None,"['Free WiFi Internet Access Included', 'Airpor...",0.660000,2563824.0,0.720000,64 reviews,NaN,NaN,NaN,NaN


In [9]:
res1.to_csv('Combined_Hotels_Summary (4).csv')